In [107]:
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime

In [3]:
def days_conversion(x):
    if "day" in x:
        x = x.split(' ')[0]
        if len(x)==1:
            return int(x)
        else:
            return int(x[x.find('-')+1])
    elif "week" in x:
        x = x.split(' ')[0]
        if len(x)==1:
            return int(x)*7
        else:
            return int(x[x.find('-')+1])*7
    elif "month" in x:
        x = x.split(' ')[0]
        if len(x)==1:
            return int(x)*30
        else:
            return int(x[x.find('-')+1])*30
    elif "year" in x:
        x = x.split(' ')[0]
        if len(x)==1:
            return int(x)*365
        else:
            return int(x[x.find('-')+1])*365
    elif "hour" in x:
        x = x.split(' ')[0]
        if len(x)==1:
            return int(x)/24
        else:
            return int(x[x.find('-')+1])/24
    elif "date" in x and re.match('^[0-9]', x):
        if 'mo' in x:
            return int(x.split(' ')[0])*30
        if 'week' in x:
            return int(x.split(' ')[0])*7
        if 'year' in x:
            return int(x.split(' ')[0])*365
        else:
            return int(x.split(' ')[0])
    else:
        return np.nan
    
        

In [11]:
combined_csv.shape

(1358, 6)

In [12]:
def period_conversion(df):
    non_null_df = df[df['PERIOD'].notnull()]
    null_df = df[df['PERIOD'].isnull()]
    # Change period column to days and consider the maximum value
    non_null_df['PERIOD'] = non_null_df['PERIOD'].apply(days_conversion)
    df = pd.concat([non_null_df,null_df],ignore_index=True)
    df["PERIOD"] = pd.to_numeric(df["PERIOD"])
    # fill period column null values by "TYPE" and "STORAGE"
    df['PERIOD'] = df.groupby(['TYPE'])['PERIOD'].transform(lambda x:x.fillna(round(np.mean(x),2)))
    return df

In [13]:
csv_files = [f for f in os.listdir() if f.endswith('.csv') and f.startswith('food')]
# 1. Merge all food.csv files
combined_csv = pd.concat([pd.read_csv(f) for f in csv_files])
# removing redundant items (by eliminating nan records)
combined_csv = combined_csv.drop_duplicates()
# separating null and non-null rows to two dataframes
combined_csv = period_conversion(combined_csv)
# Split item name and create new column
combined_csv[['ITEM','ITEM_DESC']] = combined_csv.ITEM.str.split(',', 1, expand=True)

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
combined_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TYPE       679 non-null    object 
 1   ITEM       679 non-null    object 
 2   PERIOD     679 non-null    float64
 3   DETAILS    307 non-null    object 
 4   STORAGE    679 non-null    object 
 5   ITEM_DESC  455 non-null    object 
dtypes: float64(1), object(5)
memory usage: 32.0+ KB


In [15]:
orderHeaderDonation_df = pd.read_csv("orderHeaderDonations.csv")
orderHeaderRequest_df = pd.read_csv("orderHeaderRequests.csv")

In [16]:
orderHeaderDonation_df['OrderDate'] = pd.to_datetime(orderHeaderDonation_df['OrderDate'], format='%Y-%m-%d')
orderHeaderRequest_df['OrderDate'] = pd.to_datetime(orderHeaderRequest_df ['OrderDate'], format='%Y-%m-%d')

In [17]:
orderHeaderDonation_df.OrderId = orderHeaderDonation_df.OrderId.astype(str)
orderHeaderDonation_df.PartnerId = orderHeaderDonation_df.PartnerId.astype(str)

In [18]:
orderHeaderRequest_df.OrderId = orderHeaderRequest_df.OrderId.astype(str)
orderHeaderRequest_df.CustomerId = orderHeaderRequest_df.CustomerId.astype(str)

In [19]:
orderHeaderRequest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OrderId         392 non-null    object        
 1   CustomerId      392 non-null    object        
 2   OrderTotal      392 non-null    int64         
 3   OrderDate       392 non-null    datetime64[ns]
 4   Location        392 non-null    object        
 5   CustomerName    392 non-null    object        
 6   CustomerGeoLat  392 non-null    float64       
 7   CustomerGeoLon  392 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.6+ KB


In [20]:
orderItemsDonation_df = pd.read_csv("orderItemsDonations.csv")
orderItemRequest_df = pd.read_csv("orderItemsRequests.csv")

In [21]:
orderItemsDonation_df.OrderId = orderItemsDonation_df.OrderId.astype(str)
orderItemsDonation_df.OrderItemId = orderItemsDonation_df.OrderItemId.astype(str) 
orderItemRequest_df.OrderId = orderItemRequest_df.OrderId.astype(str) 
orderItemRequest_df.OrderItemId = orderItemRequest_df.OrderItemId.astype(str) 

In [22]:
orderItemsDonation_df.columns = ['OrderId','OrderItemId','TYPE','ITEM','PERIOD','DETAILS','STORAGE','Quantity']
orderItemRequest_df.columns = ['OrderId','OrderItemId','TYPE','ITEM','PERIOD','DETAILS','STORAGE']

In [23]:
orderItemsDonation_df.head()

,OrderId,OrderItemId,TYPE,ITEM,PERIOD,DETAILS,STORAGE,Quantity
0,20051,1,DIARY,"Cheese, processed slices",1-2 months,NaN,REFRIGERATED,105
1,20051,2,DIARY,"Eggs, in shell",3-5 weeks,NaN,REFRIGERATED,120
2,20051,3,DIARY,Margarine,12 months,NaN,FROZEN,140
3,20051,4,DIARY,"Milk, plain or flavored",1 week,NaN,REFRIGERATED,135
4,20051,5,SOY,Tofu,1 week,NaN,REFRIGERATED,145


In [24]:
orderItemsDonation_df[['ITEM','ITEM_DESC']] = orderItemsDonation_df.ITEM.str.split(',', 1, expand=True)
orderItemRequest_df[['ITEM','ITEM_DESC']] = orderItemRequest_df.ITEM.str.split(',', 1, expand=True)

In [25]:
orderItemsDonation_df = period_conversion(orderItemsDonation_df)
orderItemRequest_df = period_conversion(orderItemRequest_df)

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [26]:
orderItemsDonation_df.head()

,OrderId,OrderItemId,TYPE,ITEM,PERIOD,DETAILS,STORAGE,Quantity,ITEM_DESC
0,20051,1,DIARY,Cheese,60.0,NaN,REFRIGERATED,105,processed slices
1,20051,2,DIARY,Eggs,35.0,NaN,REFRIGERATED,120,in shell
2,20051,3,DIARY,Margarine,30.0,NaN,FROZEN,140,None
3,20051,4,DIARY,Milk,7.0,NaN,REFRIGERATED,135,plain or flavored
4,20051,5,SOY,Tofu,7.0,NaN,REFRIGERATED,145,None


In [27]:
combined_csv.to_csv("food_data_cleaned.csv", index=False)
orderHeaderRequest_df.to_csv('orderHeaderRequests_cleaned.csv',index=False)
orderHeaderDonation_df.to_csv('orderHeaderDonations_cleaned.csv',index=False)
orderItemsDonation_df.to_csv("orderItemsDonation_cleaned.csv", index=False)
orderItemRequest_df.to_csv("orderItemRequest_cleaned.csv", index=False)

In [8]:
ohr = pd.read_csv('orderHeaderRequests_cleaned.csv')
ohd = pd.read_csv('orderHeaderDonations_cleaned.csv')
oid = pd.read_csv("orderItemsDonation_cleaned.csv")
oir = pd.read_csv("orderItemRequest_cleaned.csv")

In [3]:
ohd.head()

,OrderId,PartnerId,OrderTotal,OrderDate,PartnerName,PartnerGeoLat,PartnerGeoLon
0,20051,190050,2000,2021-01-11,FoodieLand,-27.49079,153.06799
1,20052,190050,2000,2021-01-12,FoodieLand,-27.49079,153.06799
2,20053,190050,2000,2021-01-13,FoodieLand,-27.49079,153.06799
3,20054,190050,2000,2021-01-14,FoodieLand,-27.49079,153.06799
4,20055,190050,2000,2021-01-15,FoodieLand,-27.49079,153.06799


In [12]:
oid.head(20)

,OrderId,OrderItemId,TYPE,ITEM,PERIOD,DETAILS,STORAGE,Quantity,ITEM_DESC
0,20051,1,DIARY,Cheese,60.00,NaN,REFRIGERATED,105,processed slices
1,20051,2,DIARY,Eggs,35.00,NaN,REFRIGERATED,120,in shell
2,20051,3,DIARY,Margarine,30.00,NaN,FROZEN,140,NaN
3,20051,4,DIARY,Milk,7.00,NaN,REFRIGERATED,135,plain or flavored
4,20051,5,SOY,Tofu,7.00,NaN,REFRIGERATED,145,NaN
5,20051,6,PROCESSED_MEAT,Bacon,7.00,NaN,REFRIGERATED,140,NaN
6,20051,7,PROCESSED_MEAT,Ham,270.00,NaN,REFRIGERATED,110,"canned (""keep refrigerated"" label)"
7,20051,8,FRUIT,Apples,2.00,NaN,SHELF,130,NaN
8,20051,9,FRUIT,Bananas,25.46,NaN,SHELF,100,NaN
9,20051,10,VEGETABLES,Tomatoes,39.50,NaN,SHELF,125,NaN


In [5]:
ohr.head()

,OrderId,CustomerId,OrderTotal,OrderDate,Location,CustomerName,CustomerGeoLat,CustomerGeoLon
0,50050,90050,200,2021-01-10,AGA North,Mark Burton,-27.54249,153.09159
1,50051,90050,200,2021-01-20,AGA North,Mark Burton,-27.54249,153.09159
2,50052,90050,200,2021-01-25,AGA North,Mark Burton,-27.54249,153.09159
3,50053,90050,200,2021-01-30,AGA North,Mark Burton,-27.54249,153.09159
4,50054,90050,200,2021-02-06,AGA North,Mark Burton,-27.54249,153.09159


In [7]:
oir.head(20)

,OrderId,OrderItemId,TYPE,ITEM,PERIOD,DETAILS,STORAGE,ITEM_DESC
0,50050,2,SHELF_ITEM,Tea,30.00,unopened,SHELF,bags
1,50050,3,BAKERY,Pies,240.00,NaN,FROZEN,mincemeat
2,50050,4,SHELF_ITEM,Marshmallows,120.00,unopened,SHELF,marshmallow creme
3,50050,5,BAKERY,Doughnuts,7.00,G,REFRIGERATED,glazed or cake
4,50050,6,DIARY,Cream,120.00,NaN,FROZEN,half-and-half
5,50050,7,SMOKEDFISH,Salmon,60.00,NaN,FROZEN,"whitefish, cold-smoked"
6,50050,8,SHELF_ITEM,Milk,5.00,opened,REFRIGERATED,canned evaporated
7,50050,9,DIARY,Milk,90.00,NaN,FROZEN,plain or flavored
8,50050,10,SHELF_ITEM,Extracts,1095.00,unopened,SHELF,"vanilla, lemon, etc."
9,50050,11,BAKERY,Rolls,36.62,NaN,SHELF,"yeast, partially baked"


In [89]:
donor_data = pd.merge(ohd,oid,on='OrderId')
consumer_data = pd.merge(ohr,oir,on='OrderId')

In [14]:
donor_data.head()

,OrderId,PartnerId,OrderTotal,OrderDate,PartnerName,PartnerGeoLat,PartnerGeoLon,OrderItemId,TYPE,ITEM,PERIOD,DETAILS,STORAGE,Quantity,ITEM_DESC
0,20051,190050,2000,2021-01-11,FoodieLand,-27.49079,153.06799,1,DIARY,Cheese,60.0,NaN,REFRIGERATED,105,processed slices
1,20051,190050,2000,2021-01-11,FoodieLand,-27.49079,153.06799,2,DIARY,Eggs,35.0,NaN,REFRIGERATED,120,in shell
2,20051,190050,2000,2021-01-11,FoodieLand,-27.49079,153.06799,3,DIARY,Margarine,30.0,NaN,FROZEN,140,NaN
3,20051,190050,2000,2021-01-11,FoodieLand,-27.49079,153.06799,4,DIARY,Milk,7.0,NaN,REFRIGERATED,135,plain or flavored
4,20051,190050,2000,2021-01-11,FoodieLand,-27.49079,153.06799,5,SOY,Tofu,7.0,NaN,REFRIGERATED,145,NaN


In [125]:
start_date = '2021-01-11'
end_date = '2021-04-06'
donor_data = donor_data[(donor_data['OrderDate']>=start_date)&(donor_data['OrderDate']<=end_date)]
consumer_data = consumer_data[(consumer_data['OrderDate']>=start_date)&(consumer_data['OrderDate']<=end_date)]
supply_total_df = donor_data.groupby('OrderDate').OrderTotal.sum()
demand_total_df = consumer_data.groupby('OrderDate').OrderTotal.sum()
idx = pd.date_range(start_date,end_date)
demand_total_df.index = pd.DatetimeIndex(demand_total_df.index)
demand_total_df = demand_total_df.reindex(idx, fill_value=0)
supply_total_df = supply_total_df.reset_index()
demand_total_df = demand_total_df.reset_index()
demand_total_df.columns = ['OrderDate','OrderTotal']
supply_total_df["OrderDate"] = supply_total_df["OrderDate"].apply(lambda x: datetime.strptime(x,"%Y-%m-%d"))
demand_total_df["OrderDate"] = demand_total_df["OrderDate"].apply(lambda x: x.date())

In [126]:
supply_total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   OrderDate   86 non-null     datetime64[ns]
 1   OrderTotal  86 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.5 KB


In [128]:
demand_total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   OrderDate   86 non-null     object
 1   OrderTotal  86 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


In [133]:
import seaborn as sns

/home/ujwalitha/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [137]:
cnt=0
stock = 0
demand = 0
li = []
food_wastage = 0
for i in range(86):
    if (supply_total_df.iloc[i]['OrderDate']==demand_total_df.iloc[i]['OrderDate']):
        stock = stock+supply_total_df.iloc[i]["OrderTotal"]
        demand = demand + demand_total_df.iloc[i]["OrderTotal"]
        li.append(stock-demand)
        cnt=cnt+1
print(cnt)
    

86


In [140]:
min(li), max(li)

(150800, 34324200)

In [69]:
min(supply_total_df.shape[0],consumer_data.shape[0])

86

In [167]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = max(li),
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Food Wastage on daily basis (in terms of Items)",'font': {'size': 15}},
    gauge = {
        'bar': {'color': "red"}},
))
fig.update_layout(
    autosize=False,
    width=500,
    height=500)

fig.show()
